In [1]:
# for the DeepGlobe dataset (6,226 images) I reserve a 500 images for validation - 500
# for the Spacenet dataset (2,780 images) I reserve a 500 images for validation - 500
# for the WHU dataset (27,770 images)  I reserve a 500 images for validation - 1000

#

# update the number of images processed

In [1]:
import sys

sys.path.append("..")

from PATHFinder.utils.download_weights import Download_Weights


In [1]:
from train import Train
import numpy as np
validation_loss = Train(
                    pretrained=True,
                    device='cuda',
                    save_log_dir=r'C:\WORKS\Master_Thesis\Codes\logs',
                    cls_weights=np.array([1, 3], dtype=np.float32),
                    freeze_train=True,
                    phi='b5',
                    save_period=1,
                    model_name=f'Testing',
                    use_f16=False, 
                    freeze_batch_size=16,
                    unfreeze_batch_size=8,
                    total_epochs=100,
                    unfreeze_epoch=50,
                    initial_lr= 1e-4, # use 1e-4, you used 5e-5 for the last training
                    # lad = 0.4,
                    plot_train_val_loss=True,
                    images_directory=r"C:\WORKS\Master_Thesis\Train_Datasets",
                    labels_directory=r"C:\WORKS\Master_Thesis\Train_Datasets",
                    early_stopping_epochs=5,
                    no_training=True
                    )


# Validation Loss: 0.6311, MIoU: 0.6431 - author's lad



#---- for the b5 all spacenet, WHU and deepglobe dataset
#Training Loss: 0.5609,  MIoU: 0.7488, LR: 4.9924335535884686e-05                                                             
# Validation Loss: 0.5828, MIoU: 0.7437

Environment variable is set!!!


c:\Users\Propietario\miniconda3\envs\datascience\lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\Propietario\miniconda3\envs\datascience\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\Propietario\miniconda3\envs\datascience\lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


ValueError: You set no_training parameter, so training won't occur

In [ ]:
#  - still trying to balance it = Validation Loss: 0.5602, MIoU: 0.5471
#- better one = Validation Loss: 0.5471, MIoU: 0.6198


#Validation Loss: 0.6049, MIoU: 0.7053
#0.26000  - initial width of the road
#-0.000012 = 1.34 meters

# Validation Loss: 0.5445, MIoU: 0.6335 - best UN unfroozen finetune
#https://www.youtube.com/watch?v=bNNfUhzivxk - pastor nathaneil's ministration at prayer

In [10]:
from PIL import Image
new_image = Image.new('L', [512, 512], (0))


In [7]:
import os
some_path = ['C:\\WORKS\\Master_Thesis\\Train_Datasets\\DeepGlobe\\273731_sat.jpg', 'C:\\WORKS\\Master_Thesis\\Train_Datasets\\WHU\\26072_sat.jpg', 'C:\\WORKS\\Master_Thesis\\Train_Datasets\\WHU\\13524_sat.jpg']
file_names = [os.path.splitext(os.path.basename(path))[0] for path in some_path]
for file in file_names:
    if 'sat' in file:
        print(file)

273731_sat
26072_sat
13524_sat


#### Things used while training

1. fp16 - using mixed precision for faster training to save GPU

2. num_classes is 2 - background and foreground

3. phi - which defines the backbone model used. In the code, phi b2 was used
4. pretrained - whether to use the segformer pretrained weights. In the code it was set to false, I do know why yet but I am going to check. In the code if pretrained is set to false, then the weights were intialised using something called xavier .... check the train.py file for more info
5. Image size - the image size used in the code is 512 by 512. but it seems like it can be changed and the best possible image sizes to use were defined in the segformer.py file 
6. frozen and unfrozen phase of training - this has to do with how many epochs to train with the backbone frozen. the batch size and epochs for freezing and unfreezing are specified. it was also specified in the paper. so try and check if paper and code correlates. Inital learning rate, momentum,weight decay, batch size and epochs were specified for frozen and unfroozen stages of training  
7. learning_rate_decay - cosine lr decay was used
8. optimizer  - AdamW was used in the code, Adam was used in the paper
9. save_period - how often per epochs to save the weight of the model
10. eval_flag and eval_period - how often to perform evaluation during training. This might not be necessary.
11. Dice loss and Focal loss - both were set to False, meaning both scores were not used as loss functions. This correlates with what is in the paper as binary cross entropy loss and cross entropy loss were used in the paper. Cross entropy loss was used as the segmentation loss while binary cross entropy loss was used as PCS loss
12. num_workers - the value 4 was used. But most times when I set num_workers my code doesn't run. So I will need to check why.
13. Training for multiple GPU, single GPU or CPU is specified
14. Pretrained_weights - downloading of pretrained weights if petrained is set to True. Otherwise if pretrained is set to False, it applies weight initialisation to the model. The function to define weights is in the repo.
15. Mixed_precision - if f16 is specified, the code uses Gradscaler to scale gradients, this helps with numerical stability. If f16 is set to False i.e not used, the scaler is set to None
16. Synchronised Batch Normalisation - this can only be used when you have multiGPUS
17. Distributed Mode - this is used only on multiple GPUs
18. A configuration display function which displays key configuration parameters like lr, num_classes, model, input_shape
19. In the code, a warning is set to make sure the model is trained for enough epochs especially when using optimizers like the AdamW as this type of optimizer requires longer training steps to reach convergence. So a recommend steps of about 15000 is set if an Adamw optimizer is used else if another type of optimizer the recommended steps is set to about 5000. The total number of steps the model will perform for the whole training of the model is defined as the length of the whole training data // unfreeze batch_size multiply by the unfreeze epoch *In code - wanted_step = 1.5e4 if optimizer_type == "adamw" else 0.5e4
total_step = num_train // Unfreeze_batch_size * UnFreeze_Epoch*.
20. When training the model, if freezing backbone layers is set to True, then the updating of the model's layer weights during backpropagation is set to False. This enables the pre-trained weights of the model to be retained,while training only the top layers of the model. This was done by setting requires grad to False in the train.py file -line 395
21. In line 406 to 410 a new minimum and maximum learning rate is defined for a batchsize different from the reference batchsize of 16. The reference batchsize is 16, but of course it is assumed that there might be difference s in computation constraints, so a new minimum and maximum learning rate is calculated using the reference batch size of 16. The code is from 406 to 410. The idea is that higher batch size requires larger learning rate (e.g 0.16) since there would be more samples seen by the model and when calculating gradients, it won't be small. Also, smaller batch sizes requires smaller learning rate (e.g 0.0025). The idea is to scale the learning rate proportionally to the batch size relative to a reference batch size. The max and min learning rate for the reference batch size of 16 is also specified.
22. The model is trained from an init epoch to the unfreeze epoch. This is defined as a range. The batch size used during training when the layers are freezed and when the layers are unfreezed are different. But  it is recommended that the Freeze_batch_size be 1-2 times the size of the Unfreeze_batch_size. The difference between both batch sizes shouldn't be much. In the code, default freeze batch size is set to 16, and default unfreeze batch size is set to 8. The batch size when the layers are unfrozen is smaller because it would require more gpu computation and memory to train since all the layers are unfrozen.
23. So the code was firstly designed for freezing the layers, then after running for the defined freeze epoch, the layers are then unfrozen so that the unfroozen epoch can be trained.


In [ ]:
import pandas as pd
from shapely.wkt import loads
import openeo
from tqdm import tqdm
connection = openeo.connect("openeo.dataspace.copernicus.eu")

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [10]:
connection.authenticate_oidc_client_credentials(
                             client_id='sh-46deba5e-c869-445d-acf7-9e1a0fc067fe',
                              client_secret='A2VIFN38cGykGEWRklbNDiY8McvY86kB'
                             )

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>